In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
movies_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/movies.csv",
                        usecols=['movieId','title'], dtype={'movieId':'int32','title':'str'})

In [4]:
ratings_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ratings.csv",
                        usecols=['userId', 'movieId','rating'],
                         dtype={'userId':'int32','movieId':'int32','rating':'float32'})

In [5]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [6]:
movies_df.shape

(9742, 2)

In [7]:
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [8]:
ratings_df.shape

(100836, 3)

In [9]:
df = pd.merge(ratings_df, movies_df, on = 'movieId')
df

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997)
100832,610,160527,4.5,Sympathy for the Underdog (1971)
100833,610,160836,3.0,Hazard (2005)
100834,610,163937,3.5,Blair Witch (2016)


In [10]:
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (combine_movie_rating.
groupby(by = ['title']) ['rating'].
count().
reset_index().
rename(columns = {'rating' : 'totalRatingCount'})
[['title', 'totalRatingCount']]
)
movie_ratingCount

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [11]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [12]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: totalRatingCount, dtype: float64


In [13]:
popularity_threshold = 65
rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,5,1,4.000,Toy Story (1995),215
2,7,1,4.500,Toy Story (1995),215
3,15,1,2.500,Toy Story (1995),215
4,17,1,4.500,Toy Story (1995),215


In [14]:
rating_popular_movie.shape

(32415, 5)

In [15]:
movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000
"40-Year-Old Virgin, The (2005)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.500
Ace Ventura: Pet Detective (1994),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,2.000,0.000,0.000,0.000,3.500,0.000,3.000
Ace Ventura: When Nature Calls (1995),0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,1.000,0.000,2.000,0.000,2.000,0.000,0.000


In [16]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [17]:
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000
"40-Year-Old Virgin, The (2005)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.500
Ace Ventura: Pet Detective (1994),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,2.000,0.000,0.000,0.000,3.500,0.000,3.000
Ace Ventura: When Nature Calls (1995),0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,1.000,0.000,2.000,0.000,2.000,0.000,0.000


In [18]:
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 87)

187


In [19]:
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000
"40-Year-Old Virgin, The (2005)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.500
Ace Ventura: Pet Detective (1994),0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,2.000,0.000,0.000,0.000,3.500,0.000,3.000
Ace Ventura: When Nature Calls (1995),0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,2.000,0.000,1.000,0.000,2.000,0.000,2.000,0.000,0.000


In [20]:
for i in range(0, len(distances.flatten())):
    if i ==0:
        print('Recomendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print ('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recomendations for Mummy, The (1999):

1: Men in Black (a.k.a. MIB) (1997), with distance of 0.4347991347312927:
2: Indiana Jones and the Temple of Doom (1984), with distance of 0.4494280219078064:
3: Fifth Element, The (1997), with distance of 0.5004291534423828:
4: RoboCop (1987), with distance of 0.5056631565093994:
5: X-Men (2000), with distance of 0.5135473608970642:
6: Face/Off (1997), with distance of 0.5143449306488037:
7: Star Wars: Episode I - The Phantom Menace (1999), with distance of 0.5148281455039978:
8: Spider-Man (2002), with distance of 0.5151567459106445:
9: Total Recall (1990), with distance of 0.5226969718933105:
10: Matrix, The (1999), with distance of 0.530174732208252:
11: Indiana Jones and the Last Crusade (1989), with distance of 0.5362417697906494:
12: Austin Powers: International Man of Mystery (1997), with distance of 0.5387777090072632:
13: Galaxy Quest (1999), with distance of 0.538847804069519:
14: Starship Troopers (1997), with distance of 0.54116380214